In [ ]:
%pip -q install google-genai

In [12]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
from google import genai # Sua importação original
client = genai.Client() # Sua inicialização original

MODEL_ID = "gemini-2.0-flash"

# Instalar Framework de agentes do Google
!pip install -q google-adk # Comentado, pois você já deve ter instalado

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types as genai_types_original # Sua importação e alias original
from datetime import date
import textwrap
from IPython.display import display, Markdown
# import requests # Não parece estar sendo usado
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
# NENHUMA ALTERAÇÃO NESTA FUNÇÃO
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session_id_dinamico = f"session_{os.urandom(4).hex()}" # Usando ID dinâmico
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id=session_id_dinamico)
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = genai_types_original.Content(role="user", parts=[genai_types_original.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id=session_id_dinamico, new_message=content):
        if event.is_final_response():
          if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts'):
            for part in event.content.parts:
              if hasattr(part, 'text') and part.text is not None:
                final_response += part.text
                # final_response += "\n" # Removido anteriormente, mantenho assim
    return final_response.strip() # .strip() para limpar espaços

# Função auxiliar para exibir texto formatado em Markdown no Colab
# NENHUMA ALTERAÇÃO NESTA FUNÇÃO
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [17]:
####################################################
# --- Agente 1: Coletor de DADOS de Doses --- #
####################################################
def agente_coletor_dados_doses(medicacao: str, especie: str, idade_fase_vida: str, peso_kg: float, data_de_hoje: str, concentracao_disponivel_mg_ml: str) -> str:
    coletor_dados_agent = Agent(
        name="agente_coletor_valores_doses_v5_tecnico", # Novo nome
        model=MODEL_ID,
        instruction=f"""
        Sua Persona e Missão:
        Você é um sistema de coleta de dados farmacológicos veterinários. Sua tarefa é encontrar informações específicas e retorná-las como UMA ÚNICA STRING DE TEXTO SIMPLES, com os valores separados pelo delimitador ';;'. NÃO inclua cabeçalhos de coluna. NÃO use formatação Markdown. **Utilize sempre terminologia técnica veterinária precisa.**

        Sua Tarefa Principal:
        Ao receber MEDICAÇÃO, ESPÉCIE, IDADE/FASE DE VIDA, PESO e uma CONCENTRAÇÃO DISPONÍVEL (mg/mL) (que pode ser 'N/A' ou uma string como '50 mg/mL'):
        1. Utilizar `google_search` para pesquisar doses da MEDICAÇÃO para a ESPÉCIE (foco: formulações injetáveis) e segurança para a IDADE/FASE DE VIDA.
        2. Priorizar fontes confiáveis (bulas de medicamentos veterinários, manuais de farmacologia veterinária, artigos científicos).
        3. Utilizar **terminologia técnica veterinária precisa** em todos os campos textuais (ex: "analgesia" em vez de "alívio da dor", "antipirético" em vez de "contra febre", "antiflogístico" em vez de "anti-inflamatório").
        4. Compilar os dados e apresentar a resposta EXCLUSIVAMENTE como UMA ÚNICA STRING, com os 7 valores na seguinte ordem, separados por ';;':
           1. Nome da Medicação (Princípio Ativo)
           2. Indicação Principal (use termos técnicos, ex: "analgesia e sedação", "controle de ectoparasitas")
           3. Apresentação (Concentração Injetável):
              - Se uma CONCENTRAÇÃO DISPONÍVEL (ex: '50 mg/mL') foi fornecida E É VÁLIDA (contém número e 'mg/mL'), use-a (ex: "Dipirona Sódica 50 mg/mL (concentração informada)").
              - Se a CONCENTRAÇÃO DISPONÍVEL for 'N/A', inválida, ou não fornecida, pesquise a concentração injetável mais comum e liste-a (ex: "Dipirona Sódica 500 mg/mL (concentração pesquisada)").
           4. Via de Administração (ex: IV, IM, SC)
           5. Intervalo de Dose (mg/kg)
           6. Volume a Administrar (mL):
              - CALCULE o volume usando a fórmula: (Dose_mg_kg * Peso_kg) / Concentracao_em_mg_ml.
              - **PRIORIZE a CONCENTRAÇÃO DISPONÍVEL (mg/mL) fornecida pelo usuário para este cálculo, se ela for numérica e válida.** Para isso, extraia o valor numérico da string de concentração (ex: de '50 mg/mL', use 50).
              - Se a concentração fornecida não for válida/numérica ou for 'N/A', tente usar a concentração que você pesquisou para o campo 'Apresentação'.
              - Se não for possível obter uma concentração numérica para o cálculo (nem fornecida, nem pesquisada), ou se a dose (mg/kg) não for encontrada ou não for numérica, retorne "Não calculável". Exiba o resultado com até 2 casas decimais se for um número.
           7. Segurança para Idade/Fase de Vida (para a idade/fase de vida fornecida, usando termos técnicos como "neonatos", "geriátricos", "gestantes", "lactantes" e descrevendo precauções específicas).

        Formato da Resposta (OBRIGATÓRIO - APENAS UMA STRING, VALORES SEPARADOS POR ';;'):
        [VALOR_MEDICAÇÃO];;[VALOR_INDICAÇÃO];;[VALOR_APRESENTAÇÃO];;[VALOR_VIA];;[VALOR_DOSE];;[VALOR_VOLUME];;[VALOR_SEGURANÇA_IDADE]

        Exemplo de Saída Esperada para Dipirona, Canina, Adulto 7 anos, 21kg, Concentração Disponível '50 mg/mL':
        Dipirona Sódica;;Analgesia, antipirético, antiflogístico;;Dipirona Sódica 50 mg/mL (concentração informada);;IM, IV;;25 mg/kg;;10.5 mL;;Geralmente seguro em caninos adultos; monitorar função renal e hepática em tratamentos prolongados.

        Exemplo de Saída Esperada para Ivermectina, Canina, Filhote 2 meses, 2kg, Concentração Disponível 'N/A' (assumindo que a pesquisa encontre 10 mg/mL como comum para injetável e dose de 0.2 mg/kg):
        Ivermectina;;Controle de endo e ectoparasitas (suscetíveis);;Ivermectina 10 mg/mL (concentração pesquisada);;SC;;0.2 mg/kg;;0.04 mL;;Contraindicado em filhotes com menos de 6 semanas. Cautela em raças sensíveis (Collies, etc.) devido ao gene MDR1. Avaliar risco/benefício.

        Instruções para Preenchimento dos Valores:
        - Se dados cruciais faltarem para um campo, use "Informação não encontrada" ou "N/A" como o valor para aquele campo.
        - Sua resposta é APENAS a string com os valores separados por ';;'.
        """,
        description="Coleta dados de dose (usando concentração fornecida se disponível) e retorna uma string com valores separados por ';;', utilizando terminologia técnica veterinária.",
        tools=[google_search]
    )
    entrada_para_agente = (
        f"Medicação: {medicacao}\n"
        f"Espécie: {especie}\n"
        f"Idade/Fase de Vida: {idade_fase_vida}\n"
        f"Peso (kg): {peso_kg}\n"
        f"Concentração Disponível (mg/mL): {concentracao_disponivel_mg_ml}\n"
        f"Data de Referência para pesquisa: {data_de_hoje}"
    )
    valores_doses_str = call_agent(coletor_dados_agent, entrada_para_agente)
    return valores_doses_str.strip()

In [20]:
############################################################
# --- Agente 2: Coletor de DADOS de Farmaco/Toxico --- #
############################################################
def agente_coletor_dados_farm_tox(medicacao_principal: str, especie_paciente: str, idade_fase_vida_paciente: str, peso_paciente_kg: float) -> str:
    coletor_dados_ft_agent = Agent(
        name="agente_coletor_valores_farm_tox_v4_tecnico", # Novo nome
        model=MODEL_ID,
        instruction=f"""
        Sua Persona e Missão:
        Você é um sistema de coleta de dados farmacológicos e toxicológicos veterinários. Sua tarefa é encontrar informações específicas e retorná-las como UMA ÚNICA STRING DE TEXTO SIMPLES, com os valores separados pelo delimitador ';;'. NÃO inclua cabeçalhos de coluna. NÃO use formatação Markdown. **Utilize sempre terminologia técnica veterinária precisa.**

        Sua Tarefa Principal:
        A partir da MEDICAÇÃO PRINCIPAL, ESPÉCIE, IDADE/FASE DE VIDA:
        1. Utilizar `google_search` para pesquisar informações detalhadas SOBRE A MEDICAÇÃO PRINCIPAL para a ESPÉCIE, considerando a IDADE/FASE DE VIDA.
        2. Focar em: Dose Tóxica (DL50 ou dose tóxica conhecida), Efeitos Adversos Relevantes (usar termos técnicos, ex: "êmese", "diarreia", "ataxia", "depressão do SNC", "nefrotoxicidade"), Tratamento da Intoxicação (medidas de suporte, antídotos específicos se houver), Interações Medicamentosas de Risco (potencialização de toxicidade, antagonismo), Sinergismo Farmacológico Benéfico (associações que melhoram eficácia ou reduzem efeitos adversos).
        3. Priorizar fontes confiáveis (manuais de toxicologia veterinária, farmacologia veterinária, bases de dados de medicamentos).
        4. Utilizar **termos técnicos e precisos** em todas as descrições. Por exemplo, em vez de "vômito", use "êmese"; em vez de "convulsão", use "atividade convulsiva" ou "convulsão".
        5. Compilar os dados e apresentar a resposta EXCLUSIVAMENTE como UMA ÚNICA STRING, com os 6 valores na seguinte ordem, separados por ';;':
           1. Nome da Medicação (Princípio Ativo)
           2. Dose Tóxica (Espécie, Idade se relevante, ex: "DL50 oral aguda em ratos: X mg/kg", "Dose tóxica em cães: >Y mg/kg")
           3. Efeitos Adversos Relevantes (Considerar Idade/Fase de Vida; usar termos técnicos)
           4. Tratamento Intoxicação (Descrever condutas técnicas, ex: "Fluidoterapia intravenosa, carvão ativado, diazepam para controle de convulsões")
           5. Interações Medicamentosas de Risco (Descrever o mecanismo se possível, ex: "AINEs: aumento do risco de ulceração gastrintestinal")
           6. Interações Sinergéticas Benéficas (Descrever o benefício, ex: "Com opioides: potenciação analgésica")

        Formato da Resposta (OBRIGATÓRIO - APENAS UMA STRING, VALORES SEPARADOS POR ';;'):
        [VALOR_MEDICAÇÃO];;[VALOR_DOSE_TOXICA];;[VALOR_EFEITOS_ADVERSOS];;[VALOR_TRATAMENTO];;[VALOR_INTERACOES_RISCO];;[VALOR_SINERGIA]

        Exemplo de Saída Esperada para Dipirona:
        Dipirona Sódica;;DL50 oral em ratos: ~3000 mg/kg. Em cães, doses >100 mg/kg podem causar salivação e êmese. Toxicidade significativa >500-600 mg/kg.;;Êmese, diarreia, dor abdominal, hipotensão (raro: agranulocitose, leucopenia em uso crônico).;;Fluidoterapia de suporte, lavagem gástrica (se ingestão recente e massiva), carvão ativado. Monitoramento hematológico em casos de suspeita de discrasia sanguínea.;;Com AINEs: ↑risco de lesão gastrintestinal e renal. Com clorpromazina: ↑risco de hipotermia grave.;;Com opioides (ex: Tramadol): sinergismo analgésico.

        Instruções para Preenchimento dos Valores:
        - Se informação não encontrada para um campo, use "Dados insuficientes" ou "N/A" como o valor para aquele campo.
        - Sua resposta é APENAS a string com os valores separados por ';;'.
        """,
        description="Coleta dados farmaco/toxico e retorna uma string com valores separados por ';;', utilizando terminologia técnica veterinária.",
        tools=[google_search]
    )
    entrada_para_agente = f"Medicação Principal: {medicacao_principal}\nEspécie do Paciente: {especie_paciente}\nIdade/Fase de Vida do Paciente: {idade_fase_vida_paciente}\nPeso do Paciente (kg): {peso_paciente_kg}"
    valores_farm_tox_str = call_agent(coletor_dados_ft_agent, entrada_para_agente)
    return valores_farm_tox_str.strip()

In [21]:
#############################################################
# --- Agente 3: Conselheiro Clínico Farmacológico --- #
#############################################################
def agente_conselheiro_clinico(
    medicacoes_consultadas_ctx: list,
    especie_paciente_ctx: str,
    idade_fase_vida_ctx: str,
    peso_paciente_kg_ctx: float,
    tabela_doses_md_str: str,
    tabela_farm_tox_md_str: str
) -> str:
    conselheiro_agent = Agent(
        name="agente_analista_farmacologico_ia_v3_tecnico", # Novo nome
        model=MODEL_ID,
        instruction=f"""
        Sua Persona e Missão:
        Você é um **assistente de IA especializado em análise farmacológica e toxicológica veterinária**. Sua função é atuar como uma ferramenta de suporte à decisão clínica para médicos veterinários, fornecendo uma análise técnica dos dados apresentados. Sua linguagem deve ser precisa, objetiva e utilizar **TERMINOLOGIA VETERINÁRIA TÉCNICA** de forma consistente (por exemplo, use 'êmese' ao invés de 'vômito', 'dispneia' ao invés de 'dificuldade para respirar', 'taquipneia' ao invés de 'respiração rápida', 'hipotensão' ao invés de 'pressão baixa', 'hipertermia' ao invés de 'febre alta', 'atividade convulsiva' ao invés de 'ataques', 'nefrotoxicidade' em vez de 'dano aos rins', 'hepatotoxicidade' em vez de 'dano ao fígado'). **Você NÃO é um médico veterinário, mas sim uma ferramenta de análise.**

        Sua tarefa é analisar criticamente as informações farmacológicas apresentadas em DUAS tabelas (Doses e Farmaco/Toxico) para a(s) medicação(ões) [{', '.join(medicacoes_consultadas_ctx) if isinstance(medicacoes_consultadas_ctx, list) else medicacoes_consultadas_ctx}] em um(a) paciente da espécie "[{especie_paciente_ctx}]", com IDADE/FASE DE VIDA de "[{idade_fase_vida_ctx}]" e peso aproximado de "[{peso_paciente_kg_ctx}] kg".
        Com base nessa análise, seu objetivo é fornecer **uma análise técnica dos dados, destacar pontos de atenção críticos (especialmente relacionados à idade/fase de vida, adequação da dose/volume calculados com base na apresentação, e possíveis interações medicamentosas de risco ou sinergismos benéficos entre múltiplas drogas, se aplicável) e identificar potenciais otimizações terapêuticas com base nos dados fornecidos, apresentando isso como um relatório analítico em LINGUAGEM TÉCNICA.**

        As tabelas recebidas contêm:
        1.  **Tabela de Informações de Dose:** Medicação, Indicação, apresentação (concentração injetável, indicando se foi fornecida ou pesquisada), via, intervalo de dose, volume a administrar (calculado com base na apresentação) e avaliação de segurança para idade/fase de vida.
        2.  **Tabela de Informações Toxicológicas e Farmacodinâmicas:** Medicação, Dose tóxica, efeitos adversos, tratamento intoxicação, interações medicamentosas de risco e sinergismo benéfico.

        Seu Foco ao Revisar (para formular seu relatório analítico TÉCNICO):
        1.  **Consistência e Potenciais Riscos (Considerando Idade e Apresentação):** As informações para cada fármaco nas tabelas são consistentes com dados farmacológicos conhecidos? O volume a administrar é clinicamente razoável dada a apresentação (concentração) e o peso do animal (ex: volumes excessivos para administração única, volumes diminutos que dificultam a precisão)? Há riscos evidentes baseados na idade/fase de vida (neonatos, pediátricos, geriátricos, gestantes, lactantes), dose tóxica, ou efeitos adversos listados (ex: nefrotoxicidade em paciente com doença renal preexistente inferida pela idade)? A apresentação (concentração) parece adequada para a dose e o animal?
        2.  **Interações Medicamentosas (se múltiplas drogas):** Com base nas informações das tabelas ("Interações Medicamentosas de Risco" e "Interações Sinergéticas Benéficas"), há interações farmacodinâmicas ou farmacocinéticas significativas ou potenciais entre as medicações consultadas? Descreva o possível mecanismo e consequência clínica.
        3.  **Relevância Clínica e Pontos de Alerta (Foco na Idade, Dose, Volume e Protocolo Geral):** Com base nas informações de "Segurança para Idade/Fase de Vida" e "Efeitos Adversos Relevantes", existem alertas específicos para otimizar a segurança terapêutica? A dose e o volume calculado (baseado na apresentação) parecem apropriados ou levantam questões (ex: subdoses ou superdoses aparentes, risco de reações adversas dose-dependentes)?
        4.  **Gestão de Riscos e Segurança Farmacoterapêutica (baseado nos dados):** Quais riscos toxicológicos específicos são identificados nas tabelas? As condutas para "Tratamento Intoxicação" listadas são relevantes para os efeitos adversos descritos e factíveis no cenário clínico?

        Formato da SUA Resposta (RELATÓRIO ANALÍTICO TÉCNICO):
        - **Inicie SEMPRE sua resposta com:** "Como um assistente de IA especializado, segue a análise farmacotécnica dos dados fornecidos:"
        - Se, com base na análise dos dados fornecidos, tudo estiver aparentemente adequado:
          "ANÁLISE FARMACOLÓGICA PRELIMINAR: Os dados para [{', '.join(medicacoes_consultadas_ctx) if isinstance(medicacoes_consultadas_ctx, list) else medicacoes_consultadas_ctx}] em paciente {especie_paciente_ctx} ({idade_fase_vida_ctx}, {peso_paciente_kg_ctx} kg), conforme apresentados nas tabelas, demonstram consistência farmacológica e ausência de alertas críticos imediatos. Recomenda-se a revisão e validação final pelo Médico Veterinário responsável."

        - Caso identifique pontos de atenção, inconsistências nos dados fornecidos, ou necessite de detalhamento com base na análise das tabelas:
            Formule um relatório técnico, utilizando **linguagem veterinária técnica**. Se múltiplas drogas, aborde interações com base nas informações das tabelas.
            Exemplo para uma droga:
            "Analisando os dados farmacológicos de {medicacoes_consultadas_ctx if not isinstance(medicacoes_consultadas_ctx, list) else medicacoes_consultadas_ctx[0]} para paciente {especie_paciente_ctx} ({idade_fase_vida_ctx}, {peso_paciente_kg_ctx} kg) conforme tabelas:\\n- **Análise de Dose e Farmacocinética:** [Detalhe técnico da análise dos dados de dose, volume (considerando a praticidade e precisão da administração), apresentação (concentração), e segurança para idade/fase de vida. Ex: 'O volume de X mL calculado para a concentração de Y mg/mL parece adequado para administração IM. A dose de Z mg/kg está dentro do intervalo terapêutico preconizado para a espécie, porém, em pacientes pediátricos, recomenda-se iniciar com a menor dose eficaz devido à imaturidade metabólica hepática.'].\\n- **Análise Farmacodinâmica e Toxicológica:** [Detalhe técnico da análise dos dados de efeitos adversos (ex: 'Potencial para êmese e diarreia como efeitos gastrintestinais comuns.'), interações listadas (ex: 'A coadministração com fármacos X pode exacerbar o risco de nefrotoxicidade.')]."

            Exemplo para múltiplas drogas:
            "Analisando o protocolo farmacológico com base nos dados de [{', '.join(medicacoes_consultadas_ctx)}] para paciente {especie_paciente_ctx} ({idade_fase_vida_ctx}, {peso_paciente_kg_ctx} kg) conforme tabelas:\\n- **{medicacoes_consultadas_ctx[0] if len(medicacoes_consultadas_ctx)>0 else 'Fármaco 1'} (análise individual):** [Análise técnica dos dados específicos para este fármaco, incluindo dose, apresentação, volume, segurança, e principais efeitos adversos/toxicológicos listados].\\n- **{medicacoes_consultadas_ctx[1] if len(medicacoes_consultadas_ctx)>1 else 'Fármaco 2'} (análise individual):** [Análise técnica dos dados específicos para este fármaco].\\n- **Análise de Interações Medicamentosas Potenciais (com base nas tabelas):** [Detalhe da análise das interações farmacodinâmicas ou farmacocinéticas listadas nas tabelas entre {medicacoes_consultadas_ctx[0] if len(medicacoes_consultadas_ctx)>0 else ''} e {medicacoes_consultadas_ctx[1] if len(medicacoes_consultadas_ctx)>1 else ''}. Ex: 'A tabela indica um risco de interação entre Fármaco A e Fármaco B, podendo resultar em [efeito, ex: prolongamento do intervalo QT]. Considerar monitoramento eletrocardiográfico ou ajuste de dose.']."

        - **Finalize SEMPRE sua resposta, em uma nova linha separada, com:**
          "Lembre-se: Este é um relatório analítico gerado por uma ferramenta de IA e não substitui o julgamento clínico, a experiência profissional e a responsabilidade final de um médico veterinário qualificado. As informações aqui contidas devem ser cuidadosamente avaliadas e validadas pelo profissional responsável antes de qualquer decisão clínica."

        Utilize `google_search` APENAS SE ESTRITAMENTE NECESSÁRIO para verificar alguma inconsistência FLAGRANTE nos dados das tabelas fornecidas, mas sua análise primária deve ser sobre os dados recebidos.
        Sua resposta NÃO é uma tabela, mas sim um relatório técnico dissertativo.
        """,
        description="Fornece análise técnica farmacológica baseada em dados, como um assistente de IA, utilizando linguagem veterinária técnica.",
        tools=[google_search]
    )

    meds_str_param = medicacoes_consultadas_ctx
    if isinstance(medicacoes_consultadas_ctx, list):
        meds_str_param = ', '.join(medicacoes_consultadas_ctx)

    entrada_para_agente = (
        f"DADOS PARA ANÁLISE E PARECER FARMACOLÓGICO ASSISTIDO POR IA:\n"
        f"Medicação(ões) Principal(ais) para análise: {meds_str_param}\n"
        f"Espécie do Paciente: {especie_paciente_ctx}\n"
        f"Idade/Fase de Vida do Paciente: {idade_fase_vida_ctx}\n"
        f"Peso do Paciente (kg): {peso_paciente_kg_ctx}\n\n"
        f"TABELA 1 RECEBIDA (INFORMAÇÕES DE DOSE - Markdown):\n{tabela_doses_md_str}\n\n"
        f"TABELA 2 RECEBIDA (INFORMAÇÕES TOXICOLÓGICAS E FARMACODINÂMICAS - Markdown):\n{tabela_farm_tox_md_str}\n\n"
        f"Por favor, analise criticamente estas informações conforme suas instruções e forneça seu relatório analítico em linguagem técnica."
    )
    feedback_do_conselheiro = call_agent(conselheiro_agent, entrada_para_agente)
    return feedback_do_conselheiro

In [22]:
# --- Fluxo Principal de Execução --- #
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Iniciando o Sistema de Consulta Farmacológica Veterinária ")
print("="*60)

try:
    especie_ctx_global = input("🐾 Digite a ESPÉCIE do paciente (ex: Canina, Felina): ").strip()
    idade_input_str_global = input("📅 Digite a IDADE do paciente (ex: 2 meses, 1 ano, adulto, neonato, geriátrico): ").strip() # Adicionado exemplos
    peso_input_str_global = input("⚖️ Digite o PESO do paciente em kg (ex: 21): ").strip()
    medicacoes_input_str = input("💊 Digite a(s) MEDICAÇÃO(ÕES) (princípios ativos, separados por vírgula): ").strip() # Adicionado 'princípios ativos'

    if not especie_ctx_global or not idade_input_str_global or not peso_input_str_global or not medicacoes_input_str:
        raise ValueError("Um ou mais campos de entrada estão vazios.")

    if "kg" in peso_input_str_global.lower():
        peso_kg_ctx_global = float(peso_input_str_global.lower().replace("kg", "").strip())
    else:
        peso_kg_ctx_global = float(peso_input_str_global.strip())
    idade_fase_vida_ctx_global = idade_input_str_global
    lista_medicacoes_consultadas = [med.strip() for med in medicacoes_input_str.split(',') if med.strip()]
    if not lista_medicacoes_consultadas:
        raise ValueError("Nenhuma medicação fornecida.")

    # ---- NOVA ETAPA: Coleta de Concentrações Disponíveis ----
    concentracoes_disponiveis = {}
    print("\n--- 📝 Por favor, informe as concentrações disponíveis (mg/mL) ---")
    for med in lista_medicacoes_consultadas:
        while True:
            conc_input = input(f"Qual a CONCENTRAÇÃO de '{med}' disponível (ex: 50 mg/mL, ou deixe em branco/N/A se não souber): ").strip()
            if not conc_input or conc_input.lower() == "n/a":
                concentracoes_disponiveis[med] = "N/A"
                break
            if "mg/ml" in conc_input.lower() and any(char.isdigit() for char in conc_input):
                concentracoes_disponiveis[med] = conc_input
                break
            else:
                print("Formato inválido. Use 'X mg/mL' (ex: '50 mg/mL') ou deixe em branco/N/A.")
    # ---------------------------------------------------------

    print("\n======================================================")


    # Estruturas para armazenar as linhas de dados FORMATADAS EM MARKDOWN para as tabelas finais
    linhas_markdown_tabela_doses = []
    linhas_markdown_tabela_farm_tox = []

    # Cabeçalhos e separadores são definidos uma vez
    cabecalho_tabela_doses = "| Medicação | Indicação Principal | Apresentação (Concentração Injetável) | Via de Administração | Intervalo de Dose (mg/kg) | Volume a Administrar (mL) | Segurança para Idade/Fase de Vida |"
    separador_tabela_doses = "| :-------- | :------------------ | :------------------------------------ | :------------------- | :------------------------ | :------------------------ | :------------------------------------ |"

    cabecalho_tabela_farm_tox = "| Medicação | Dose Tóxica (Espécie, Idade relevante) | Efeitos Adversos Relevantes (Considerar Idade) | Tratamento Intoxicação | Interações Medicamentosas de Risco | Interações Sinergéticas Benéficas |"
    separador_tabela_farm_tox = "| :-------- | :----------------------------------------- | :--------------------------------------------- | :----------------------- | :------------------------------------ | :--------------------------------------- |"

    linhas_markdown_tabela_doses.append(cabecalho_tabela_doses)
    linhas_markdown_tabela_doses.append(separador_tabela_doses)
    linhas_markdown_tabela_farm_tox.append(cabecalho_tabela_farm_tox)
    linhas_markdown_tabela_farm_tox.append(separador_tabela_farm_tox)

    houve_erro_geral = False

    print(f"\n⏳ Processando informações... Por favor, aguarde.")

    for medicacao_individual_ctx in lista_medicacoes_consultadas:
        print(f"  Coletando dados para: {medicacao_individual_ctx}...")
        erro_nesta_droga = False
        concentracao_para_agente = concentracoes_disponiveis.get(medicacao_individual_ctx, "N/A")

        # --- Agente 1: Coletor de DADOS de Doses ---
        valores_doses_str = agente_coletor_dados_doses(
            medicacao_individual_ctx,
            especie_ctx_global,
            idade_fase_vida_ctx_global,
            peso_kg_ctx_global,
            data_de_hoje,
            concentracao_para_agente
        )
        if valores_doses_str.strip().startswith("ERRO"):
            print(f"    ERRO do Agente 1 para {medicacao_individual_ctx}: {valores_doses_str}")
            linhas_markdown_tabela_doses.append(f"| {medicacao_individual_ctx} | ERRO AO OBTER DADOS DE DOSE ({valores_doses_str}) | - | - | - | - | - |")
            erro_nesta_droga = True
            houve_erro_geral = True
        else:
            valores_t1 = valores_doses_str.split(';;')
            if len(valores_t1) == 7:
                linhas_markdown_tabela_doses.append(f"| {valores_t1[0].strip()} | {valores_t1[1].strip()} | {valores_t1[2].strip()} | {valores_t1[3].strip()} | {valores_t1[4].strip()} | {valores_t1[5].strip()} | {valores_t1[6].strip()} |")
            else:
                print(f"    AVISO: Agente 1 para {medicacao_individual_ctx} retornou formato inesperado ({len(valores_t1)} campos): {valores_doses_str}")
                linhas_markdown_tabela_doses.append(f"| {medicacao_individual_ctx} | ERRO DE FORMATO (AG1) | - | - | - | - | - |")
                erro_nesta_droga = True
                houve_erro_geral = True

        # --- Agente 2: Coletor de DADOS de Farmaco/Toxico ---
        if not erro_nesta_droga:
            valores_farm_tox_str = agente_coletor_dados_farm_tox(
                medicacao_individual_ctx, especie_ctx_global, idade_fase_vida_ctx_global, peso_kg_ctx_global
            )
            if valores_farm_tox_str.strip().startswith("ERRO"):
                print(f"    ERRO do Agente 2 para {medicacao_individual_ctx}: {valores_farm_tox_str}")
                linhas_markdown_tabela_farm_tox.append(f"| {medicacao_individual_ctx} | ERRO AO OBTER DADOS FARMACO/TOXICO ({valores_farm_tox_str}) | - | - | - | - |")
                houve_erro_geral = True
            else:
                valores_t2 = valores_farm_tox_str.split(';;')
                if len(valores_t2) == 6:
                    linhas_markdown_tabela_farm_tox.append(f"| {valores_t2[0].strip()} | {valores_t2[1].strip()} | {valores_t2[2].strip()} | {valores_t2[3].strip()} | {valores_t2[4].strip()} | {valores_t2[5].strip()} |")
                else:
                    print(f"    AVISO: Agente 2 para {medicacao_individual_ctx} retornou formato inesperado ({len(valores_t2)} campos): {valores_farm_tox_str}")
                    linhas_markdown_tabela_farm_tox.append(f"| {medicacao_individual_ctx} | ERRO DE FORMATO (AG2) | - | - | - | - |")
                    houve_erro_geral = True
        else:
             linhas_markdown_tabela_farm_tox.append(f"| {medicacao_individual_ctx} | DADOS NÃO COLETADOS (ERRO AG1) | - | - | - | - |")


    # --- Exibição das Tabelas Consolidadas ---
    print("\n\n======================================================")
    print("--- 📄 RELATÓRIO FARMACOLÓGICO CONSOLIDADO ---")
    print("======================================================")

    contexto_paciente_str_display = f"**Medicações Consultadas:** {', '.join(lista_medicacoes_consultadas)}\n**Espécie:** {especie_ctx_global} | **Idade/Fase de Vida:** {idade_fase_vida_ctx_global} | **Peso:** {peso_kg_ctx_global} kg\n"
    concentracoes_str_display_list = []
    for med_name, conc_val in concentracoes_disponiveis.items():
        if conc_val != "N/A":
            concentracoes_str_display_list.append(f"{med_name}: {conc_val}")
        else:
            concentracoes_str_display_list.append(f"{med_name}: Concentração não informada/pesquisada pelo sistema")

    if concentracoes_str_display_list:
         contexto_paciente_str_display += "**Concentrações Informadas/Utilizadas:** " + " | ".join(concentracoes_str_display_list) + "\n"

    display(Markdown(contexto_paciente_str_display))

    print("\n**Tabela 1: Informações de Dose**")
    tabela_doses_final_str = "\n".join(linhas_markdown_tabela_doses)
    display(Markdown(tabela_doses_final_str))

    print("\n**Tabela 2: Informações Toxicológicas e Farmacodinâmicas**")
    tabela_farm_tox_final_str = "\n".join(linhas_markdown_tabela_farm_tox)
    display(Markdown(tabela_farm_tox_final_str))
    print("--------------------------------------------------------------")

    # --- Agente 3 (Conselheiro Clínico) ---
    if not houve_erro_geral:
        revisar = input(f"\n❓ Deseja o parecer do Conselheiro Clínico sobre o relatório consolidado? (s/n): ").strip().lower()
        if revisar == 's':
            print(f"\n⏳ Solicitando parecer do Conselheiro Clínico... Por favor, aguarde.")
            parecer_conselheiro = agente_conselheiro_clinico(
                lista_medicacoes_consultadas,
                especie_ctx_global,
                idade_fase_vida_ctx_global,
                peso_kg_ctx_global,
                tabela_doses_final_str,
                tabela_farm_tox_final_str
            )
            print(f"\n--- 🧐 Parecer do Agente 3 (Conselheiro Clínico) ---")
            display(Markdown(parecer_conselheiro))
            print("--------------------------------------------------------------")
        else:
            print(f"\nℹ️ Parecer do Conselheiro Clínico pulado.")
    else:
        print("\n⚠️ Conselheiro Clínico (Agente 3) não executado devido a erros na coleta de dados ou formato inesperado.")

    print("\n✅✅✅ Todas as consultas foram processadas! ✅✅✅")

except ValueError as e:
    print(f"❌ Erro na entrada de dados: {e}. Por favor, reinicie e tente novamente.")
except Exception as e:
    print(f"❌ Ocorreu um erro inesperado durante a execução: {e} (Tipo: {type(e).__name__})")
    traceback.print_exc()

Iniciando o Sistema de Consulta Farmacológica Veterinária 
🐾 Digite a ESPÉCIE do paciente (ex: Canina, Felina): Canina
📅 Digite a IDADE do paciente (ex: 2 meses, 1 ano, adulto, neonato, geriátrico): 3 anos
⚖️ Digite o PESO do paciente em kg (ex: 21): 21
💊 Digite a(s) MEDICAÇÃO(ÕES) (princípios ativos, separados por vírgula): Dipirona, Tramadol

--- 📝 Por favor, informe as concentrações disponíveis (mg/mL) ---
Qual a CONCENTRAÇÃO de 'Dipirona' disponível (ex: 50 mg/mL, ou deixe em branco/N/A se não souber): 500 mg/ml
Qual a CONCENTRAÇÃO de 'Tramadol' disponível (ex: 50 mg/mL, ou deixe em branco/N/A se não souber): 50 mg/ml


⏳ Processando informações... Por favor, aguarde.
  Coletando dados para: Dipirona...
  Coletando dados para: Tramadol...


--- 📄 RELATÓRIO FARMACOLÓGICO CONSOLIDADO ---


**Medicações Consultadas:** Dipirona, Tramadol
**Espécie:** Canina | **Idade/Fase de Vida:** 3 anos | **Peso:** 21.0 kg
**Concentrações Informadas/Utilizadas:** Dipirona: 500 mg/ml | Tramadol: 50 mg/ml



**Tabela 1: Informações de Dose**


| Medicação | Indicação Principal | Apresentação (Concentração Injetável) | Via de Administração | Intervalo de Dose (mg/kg) | Volume a Administrar (mL) | Segurança para Idade/Fase de Vida |
| :-------- | :------------------ | :------------------------------------ | :------------------- | :------------------------ | :------------------------ | :------------------------------------ |
| Dipirona Sódica | Analgesia, antipirético, antiflogístico | Dipirona Sódica 500 mg/mL (concentração informada) | IM, IV | 30 mg/kg | 0.13 mL | Geralmente seguro em caninos adultos; monitorar função renal e hepática em tratamentos prolongados. |
| Tramadol | Analgesia (dor moderada a severa) | Tramadol 50 mg/mL (concentração informada) | IM, IV, SC | 2-4 mg/kg | 0.84-1.68 mL | Administrar com cautela em cães com alterações prévias do sistema nervoso central ou redução do limiar convulsivo. O perfil de segurança de tramadol foi similar em pacientes adultos e pediátricos (idades de 1 a 17 anos). |


**Tabela 2: Informações Toxicológicas e Farmacodinâmicas**


| Medicação | Dose Tóxica (Espécie, Idade relevante) | Efeitos Adversos Relevantes (Considerar Idade) | Tratamento Intoxicação | Interações Medicamentosas de Risco | Interações Sinergéticas Benéficas |
| :-------- | :----------------------------------------- | :--------------------------------------------- | :----------------------- | :------------------------------------ | :--------------------------------------- |
| Dipirona Sódica | DL50 oral em ratos: ~3000 mg/kg. Em cães, doses >100 mg/kg podem causar salivação e êmese. Toxicidade significativa >500-600 mg/kg ( doses elevadas do fármaco, cerca de 100mg/kg, podem causar hepatotoxicidade) | Êmese, diarreia, dor abdominal, hipotensão (raro: agranulocitose, leucopenia em uso crônico), gastrite, úlceras, salivação excessiva, falta de apetite, anemia, lesões renais. | Fluidoterapia de suporte, lavagem gástrica (se ingestão recente e massiva), carvão ativado. Monitoramento hematológico em casos de suspeita de discrasia sanguínea. | Com AINEs: ↑risco de lesão gastrintestinal e renal. Com clorpromazina: ↑risco de hipotermia grave. Com ciclosporina: redução da concentração plasmática de ciclosporina. Não utilizar concomitantemente a dipirona com outros anti-inflamatórios como fenilbutazona e ácido acetil salicílico, pois poderá ocorrer somatória das ações tóxicas ou farmacológicas. Não utilizar concomitantemente a dipirona com barbitúricos. | Com opioides (ex: Tramadol): sinergismo analgésico. |
| Tramadol | Dose diária máxima recomendada: 16 mg/kg. A resposta individual varia dependendo da dose, idade, sensibilidade à dor e estado geral do paciente. Superdosagem: miose, vômitos, colapso cardiovascular, alterações de consciência (incluindo coma), convulsões e depressão respiratória. | Sedação, sonolência, náuseas, vômitos, constipação, sialorreia, retenção urinária. Em casos raros, hipersensibilidade e, muito raramente, convulsões (em cães com baixo limiar convulsivo). | Em caso de intoxicação: Manutenção das vias aéreas, suporte cardíaco e respiratório. Naloxona pode reverter a depressão respiratória, mas pode não reverter completamente outros efeitos. Para convulsões, administrar diazepam. Lavagem gástrica pode ser considerada se a ingestão foi recente e o animal está consciente. | Interage com depressores do SNC (potencia a depressão do SNC e respiratória). AINEs: pode aumentar o risco de convulsões. Inibidores ou indutores do CYP450 podem interferir no efeito analgésico. Não associar com agonistas/antagonistas mistos (ex: buprenorfina) pois pode diminuir o efeito analgésico. | Pode ser usado em associação com AINEs, amantadina, gabapentina e outros opioides para analgesia multimodal. Dipirona e tramadol atuam sinergicamente para analgesia. |

--------------------------------------------------------------

❓ Deseja o parecer do Conselheiro Clínico sobre o relatório consolidado? (s/n): s

⏳ Solicitando parecer do Conselheiro Clínico... Por favor, aguarde.

--- 🧐 Parecer do Agente 3 (Conselheiro Clínico) ---


Como um assistente de IA especializado, segue a análise farmacotécnica dos dados fornecidos:

Analisando o protocolo farmacológico com base nos dados de [Dipirona, Tramadol] para paciente Canina (3 anos, 21.0 kg) conforme tabelas:
- **Dipirona (análise individual):** A dose de Dipirona Sódica (30 mg/kg, IM ou IV) está dentro do intervalo terapêutico usual para analgesia e efeito antipirético em caninos. O volume a ser administrado (0.13 mL) é exequível e permite precisão na administração, dada a concentração da apresentação (500 mg/mL). A segurança para a idade do paciente (3 anos) é considerada adequada, conforme a tabela, embora se recomende o monitoramento das funções renal e hepática em tratamentos prolongados. A tabela indica um risco de hepatotoxicidade com doses elevadas (acima de 100mg/kg) devendo-se evitar doses maiores.
- **Tramadol (análise individual):** A dose de Tramadol (2-4 mg/kg, IM, IV ou SC) está de acordo com as recomendações para analgesia em cães. O volume a ser administrado (0.84-1.68 mL) é apropriado para a concentração da apresentação (50 mg/mL). A segurança do tramadol é mencionada com cautela em pacientes com histórico de alterações do sistema nervoso central ou baixo limiar convulsivo, o que requer atenção ao histórico do paciente.
- **Análise de Interações Medicamentosas Potenciais (com base nas tabelas):** A tabela indica um sinergismo benéfico entre Dipirona e Tramadol para analgesia multimodal, o que pode permitir uma redução nas doses individuais de cada fármaco, minimizando potenciais efeitos adversos. Não há interações de risco descritas nas tabelas entre Dipirona e Tramadol.

Lembre-se: Este é um relatório analítico gerado por uma ferramenta de IA e não substitui o julgamento clínico, a experiência profissional e a responsabilidade final de um médico veterinário qualificado. As informações aqui contidas devem ser cuidadosamente avaliadas e validadas pelo profissional responsável antes de qualquer decisão clínica.

--------------------------------------------------------------

✅✅✅ Todas as consultas foram processadas! ✅✅✅
